In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from utilities import* # functions in the utilities.py file

In [ ]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [ ]:
flowrate.columns

In [ ]:
flowrate = flowrate.set_index('timestamp')
flowrate.head(3)

In [ ]:
pts = pts.set_index('timestamp')
pts.head(3)

In [ ]:
combined = pts.join(flowrate, how = 'outer',  lsuffix = '_pts', rsuffix = '_fr', )

In [ ]:
combined.columns

In [ ]:
combined.head(3)

In [ ]:
combined.drop(columns = ['datetime_fr'], inplace = True)

In [ ]:
combined.columns

In [ ]:
combined.columns = ['depth_m', 'speed_mps', 'cweight_kg', 'whp_barg', 'temp_degC',
       'pressure_bara', 'frequency_hz', 'datetime', 'flow_tph']

In [ ]:
combined.columns

In [ ]:
combined.describe()

In [ ]:
combined['flow_tph'].isna().sum()

In [ ]:
combined['new_flow_tph'] = combined['flow_tph'].interpolate(method='linear')

In [ ]:
combined['new_flow_tph'].isna().sum()

In [ ]:
combined.tail()

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(24,8))

ax.scatter(combined.datetime, combined.new_flow_tph, 
    c='k', s = 5, linewidths = 0)

ax.scatter(combined.datetime, combined.flow_tph, 
    c='r', s = 30, linewidths = 0)

ax.scatter(pts.datetime, pts.cweight_kg, 
    c='b', s = 5, linewidths = 0, label = 'Raw cabel weight')

ax.scatter(remaining_nan.datetime, remaining_nan.cweight_kg, 
    c='g', s = 30, linewidths = 0, label = 'NAN cabel weight')

ax.set_ylim(0,150)

ax.set_xlabel('Time [hh:mm]')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))


start_time = pd.to_datetime('2020-12-11 09:25:00')
end_time = pd.to_datetime('2020-12-11 09:30:00')

#ax.set_xlim(start_time,end_time);

As the plot and the remaining_nan.describe() show that I have filled all the PTS values with flow rates, I will now drop the flow values that have no pts value

In [ ]:
nan_trimmed = combined.dropna()

nan_trimmed = combined[combined['depth_m'].notna()]


In [ ]:
nan_trimmed.shape 

In [ ]:
combined.shape

In [ ]:
nan_trimmed.reset_index(inplace=True)

In [ ]:
nan_trimmed.head(2)

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(24,8))

ax.scatter(combined.datetime, combined.new_flow_tph, 
    c='k', s = 5, linewidths = 0)

ax.scatter(combined.datetime, combined.flow_tph, 
    c='r', s = 30, linewidths = 0)

ax.scatter(pts.datetime, pts.cweight_kg, 
    c='b', s = 5, linewidths = 0, label = 'Raw cabel weight')

ax.scatter(nan_trimmed.datetime, nan_trimmed.cweight_kg, 
    c='g', s = 30, linewidths = 0, label = 'NAN cabel weight')

ax.set_ylim(0,150)

ax.set_xlabel('Time [hh:mm]')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))


start_time = pd.to_datetime('2020-12-11 09:25:00')
end_time = pd.to_datetime('2020-12-11 09:30:00')

#ax.set_xlim(start_time,end_time);

In [ ]:
# turning the method into a function

flowrate_f = read_flowrate(r'Data-FlowRate.xlsx')
pts_f = read_pts(r'Data-PTS.xlsx')

In [ ]:
def append_flowrate_to_pts(flowrate_df, pts_df):
    '''
    Add surface flowrate to pts data

    Note that the flowrate data is recorded at a courser time resolution than the pts data
    The function makes a linear interpolation to fill the data gaps
    Refer to bonus-combine-data.ipynb to review this method and adapt it for your own data

    Args:   flowrate and pts dataframes generated by the read_flowrate and read_pts functions

    Returns: pts dataframe with flowrate tph added
    '''
    flowrate_df = flowrate_df.set_index('timestamp')
    pts_df = pts_df.set_index('timestamp')
    combined_df = pts_df.join(flowrate_df, how = 'outer',  lsuffix = '_pts', rsuffix = '_fr')
    combined_df.drop(columns = ['datetime_fr'], inplace = True)
    combined_df.columns = ['depth_m', 'speed_mps', 'cweight_kg', 'whp_barg', 'temp_degC',
       'pressure_bara', 'frequency_hz', 'datetime', 'flow_tph']
    combined_df['interpolated_flow_tph'] = combined_df['flow_tph'].interpolate(method='linear')
    trimmed_df = combined_df[combined_df['depth_m'].notna()]
    trimmed_df.reset_index(inplace=True)
    return trimmed_df

mudged_df = append_flowrate_to_pts(flowrate_f, pts_f)

In [ ]:
mudged_df.shape

In [ ]:
mudged_df.head()